# chunk_predict
download chunks
overlap is inevitable missing cell bodies is much more dangerous

In [1]:
%pwd
sys.path.append(os.path.abspath("../segmentation"))
# to import rootID_lookup 

In [2]:
# libraries
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os

from cloudvolume import CloudVolume, view
import cc3d
from tifffile.tifffile import imwrite

import rootID_lookup as IDlook

In [3]:
# setting
cv = CloudVolume('precomputed://gs://zetta_lee_fly_vnc_001_precomputed/fanc_v4_em', use_https=True)
# loading from US server to use mip0

[ 83968 223232   4390]


In [47]:
# make grid to download chunked volumes
[X,Y,Z]=cv.mip_volume_size(0)
# [ 83968 223232   4390]
# print(cv.mip_volume_size(0))
# print(cv.mip_chunk_size(0))


step_xy = 128*2**4 # width of each chunk = x or y space between each chunk center in mip0
step_z = 256 # depth of each chunk = z space between each chunk center in mip0

start_xy = 128*2**(4-1) # first chunk center
start_z = 256*2**(-1) # first chunk center

centerX = np.arange(start_xy, X, step_xy)
centerY = np.arange(start_xy, Y, step_xy)
centerZ = np.arange(start_z, Z, step_z)

# looks okay but there can be only a few space < step/2 at the end of these sequences, causing error when making chunks
if (X - centerX[-1]) < start_xy:
    np.put(centerX, -1, X-start_xy)
else:
    centerX = np.append(centerX, X-start_xy)

if (Y - centerY[-1]) < start_xy:
    np.put(centerY, -1, Y-start_xy)
else:
    centerY = np.append(centerY, Y-start_xy)

if (Z - centerZ[-1]) < start_z:
    np.put(centerZ, -1, Z-start_z)
else:
    centerZ = np.append(centerZ, Z-start_z)

# make nx3 arrays of the chunk center coordinates
chunk_center = np.array(np.meshgrid(centerX, centerY, centerZ)).T.reshape(-1,3)
len(chunk_center)

83160

In [50]:
len(centerZ)

18

array([[1.02400e+03, 1.02400e+03, 1.28000e+02],
       [1.02400e+03, 1.02400e+03, 3.84000e+02],
       [1.02400e+03, 1.02400e+03, 6.40000e+02],
       ...,
       [8.29440e+04, 2.22208e+05, 3.96800e+03],
       [8.29440e+04, 2.22208e+05, 4.22400e+03],
       [8.29440e+04, 2.22208e+05, 4.26200e+03]])

In [5]:
nuclei_cv = CloudVolume(
    'precomputed://gs://ranl_scratch_zetta_30/neuroglancer/nuclei/3d528bc3def9f51e09ae55e68ee46518',
    progress=False,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True
)

[256 256  16]


In [ ]:
for chunk in gridpoints
nuclei = nuclei_cv.download_point( (52785, 86885, 1894), mip=4, size=(128, 128, 256) ) # mip0 and 4 only
# using mio4 to make it faster
# 4.3*(2**4)*128/45=196

Following steps are similar to predict.ipynb

In [ ]:
# size of entire FANC image in mip4

In [ ]:
img.cache.flush()